# App for monitoring SSH connections

## SSH Monitoring for the BIZON Z500 Deep Learning Workstation
### Introduction
Secure Shell Protocol (SSH) monitoring can be challenging for users due to the need for terminal commands, a connection setup with a username, IP address, and password. To simplify this process, we developed a user-friendly software. This application offers a more intuitive interface, consolidating all data in one place.
### Development Overview
The purpose of this software is to efficiently monitor the BIZON Z500 Deep Learning Workstation for SSH activity. It remotely monitors the workstation, eliminating the need for users to manually run commands. The software, developed using the Bash and Python programming languages and Pandas, Gradio, and SQLite libraries, automatically connects to the server, runs necessary commands, and stores data in a database. This information is then presented on a user-friendly website.
### Key Features
Key features include:

•	Automatic connection to the workstation

•	Centralized data storage

•	Regular updates to the database

•	Graphical User Interface (GUI) for easy navigation

### Development Process
During the development process, we prioritized creating the database, followed by the GUI, bash script, and Pandas data frame. The bash script facilitates automatic connection and command execution, while the Pandas data frame organizes the data before it is sent to the database. The data is then displayed using Gradio interface.
### Benefits
Benefits include:

•	Database storage for accessing historical records

•	Enhanced monitoring efficiency

•	User-friendly GUI, eliminating the need for command-line expertise

•	Easy data retrieval with a search bar and sorting options

### Conclusion
In conclusion, the primary goal of developing this software was to simplify SSH monitoring for users. This goal has been achieved by enabling remote monitoring without the need for manual command execution. With the Gradio link, users can conveniently access and review past login records.


## Install dependencies

Installs the necessary requirements for running this application. Gradio is a python library and is used for creating the webpage. More information on Gradio can be found at https://www.gradio.app/ . A fsspec upgrade is required so Gradio can run properly but it may not be needed on all machines. Pandas is a python library used for creating the dataframes. MOre information on pandas can be found at https://pandas.pydata.org/docs/ .

In [ ]:
!pip install gradio
!pip install fsspec --upgrade
!pip install pandas

This imports the libraries that are needed for running this application.

In [ ]:
import pandas as pd
import gradio as gr
import sqlite3

Bash script used to connect to the server and run the last command which gives us the information needed for monitoring the server. This currently uses ssh pass which may need to be installed if you get an error. This can be installed with this command in the cell below. When logging into your SSH you will need to change the username and IP address and have a text file with the servers password. Also the path to where you want your csv file saved should be specified.

In [ ]:
%%bash
#!/usr/bin/bash
#How to install sshpass package
#apt-get install sshpass
sshpass -f password.txt ssh user@IP Address
cd Desktop/F_Jaramillo_Project
output="$(last -s yesterday -t today)"
echo "$output" >>log.csv

Creates the pandas dataframe for later use in the application.

In [ ]:
names = ['Username', 'IP', 'LoginTime', 'LogoutTime', 'TotalTime', 'Terminal']
df = pd.read_csv('log.csv', sep=' ', names=names)
print(df.head())

Establishes a connection to the sqlite database and creates a table named Table1. The cell below should only be ran once as it will produce an error if it is ran more than once. The error is produced because the table already exists and does not need to be created again.

In [ ]:
conn = sqlite3.connect("test.db")
cur = conn.cursor()

sql = """
create table Table1
    (Username varchar(256),
    IP varchar(256),
    LoginTime varchar(256),
    LogoutTime varchar(256),
    TotalTime varchar(256),
    Terminal varchar(256)
    );
"""

cur.execute(sql)
print("database has been created")

conn.commit()
conn.close()

Pushes the information from the pandas dataframe created earlier to the newly created mysqlite database.

In [ ]:
conn = sqlite3.connect("test.db")
df.to_sql("Table1", conn,schema=None, if_exists='append', index=False, index_label=None, chunksize=6, dtype=None, method=None)
conn.close()

Converts the information stored in the database back into a datafram for use on the gradio webpage.

In [ ]:
#Converts the Database to a pandas DataFrame so it can be used on the Gradio web page
connection = sqlite3.connect("test.db")
sql = pd.read_sql_query("SELECT * FROM Table1", connection)
df = pd.DataFrame(sql, columns=["Username", "IP","LoginTime", "LogoutTime",
                     "TotalTime","Terminal"])
connection.close()
print(df)

Creates the Gradio webpage and provides 2 links. The first link being the local link and the second is a shareable link that can be sent to others that may need access.

In [ ]:
css = """
#warning textarea {background-color: #595cff;
}
"""

def filter_records(Textbox, dropdown):
    filter = df[df[dropdown] == Textbox]
    return filter

with gr.Blocks(theme=gr.themes.Base(), css=css) as first_page:
    with gr.Tabs() as pages:
        with gr.TabItem("Database", id=0) as page1:
            with gr.Row():
                gr.Textbox(scale=0, show_label=False, value='Look up', elem_id="warning")
                Textbox = gr.Textbox(scale=2, show_label=False)
                gr.Textbox(scale=0, show_label=False, value='in', elem_id="warning")
                dropdown=gr.Dropdown(["Username", "IP","LoginTime", "LogoutTime",
                             "TotalTime","Terminal"],
                             scale=2, show_label=False)
                button = gr.Button(value="Search", size='sm', scale=0)
            with gr.Row():
                df_info = gr.Dataframe(height=800, value=df,
                             headers=["Username", "IP Address","Login Time", "Logout time",
                             "Total Time","Terminal"],
                             datatype=["str", "str", "str", "str", "str", "str"],
                             row_count=10,
                             col_count=(6, "fixed"))
        with gr.TabItem("Search Results", id=1) as page2:
            gr.Label=()
            button.click(fn=filter_records, inputs=[Textbox, dropdown],outputs=[gr.Dataframe(headers=["Username", "IP Address","Login Time", "Logout time",
                                 "Total Time","Terminal"])])
first_page.launch(share=True)